In [21]:
import pandas as pd

def calculate_percentage(fee, SalePrice, stock):
    if SalePrice == 0:
        return 0
    percentage = (fee / SalePrice) * 100
    portfolio = pd.read_csv('portfolio.csv')
    
    # Update the SalePrice for the given stock on the most recent day
    portfolio.loc[(portfolio['Stock Name'] == stock) & (portfolio['Day'] == portfolio['Day'].max()), 'SalePrice'] = SalePrice
    
    # Write to the log file in append mode
    with open('Log.txt', 'a') as file:
        file.write(f"{stock}, sold at {SalePrice - fee}, fee: {fee}, percentage: {percentage}\n")
    
    # Print the log message
    print(f"{stock}, sold at {SalePrice - fee}, fee: {fee}, percentage: {percentage}")
    
    return portfolio

# Example usage
fee = 0.02
SalePrice = 76.55
stock = "AAPL"
Sale = calculate_percentage(fee, SalePrice, stock)

AAPL, sold at 76.53, fee: 0.02, percentage: 0.02612671456564337


In [22]:
Sale

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date,SalePrice
0,AAPL,0,BUY,233.400000,0.000000,0.331895,77.460000,2024-10-28,NaN
1,XOM,0,BUY,118.900000,0.000000,0.642508,76.390000,2024-10-28,NaN
2,AMZN,0,BUY,188.390000,0.000000,0.643794,121.280000,2024-10-28,NaN
3,LYV,0,BUY,118.430000,0.000000,0.421621,49.930000,2024-10-28,NaN
4,INTC,0,BUY,22.920000,0.000000,2.094785,48.010000,2024-10-28,NaN
5,DNMR,0,BUY,0.710000,0.000000,0.000000,1.820665,2024-10-28,NaN
6,SPCE,0,BUY,7.740000,0.000000,0.246548,1.910000,2024-10-28,NaN
7,AAPL,1,Sell,233.669998,0.000000,0.331895,77.553933,2024-10-29,NaN
8,XOM,1,Sell,117.279999,0.000000,0.642508,75.353379,2024-10-29,NaN
9,AMZN,1,Sell,190.830002,0.000000,0.643794,122.855248,2024-10-29,NaN


In [ ]:
portfolio = pd.read_csv('portfolio.csv')
portfolio['Cash'] = portfolio[portfolio['Day'] == portfolio['Day'].max()]['SalePrice'].sum()
